# Getting started

## Importing tensorflow and defining the two core objects Graph and Session

- graph is what collects all computations
- session is the execution environment
- there's always a (default) graph, sess apparently needs to be initialised

In [ ]:
import tensorflow as tf

def_graph = tf.get_default_graph() # doesn't do anything, but nice to know we can touch the graph
sess = tf.Session()


# 'Construction' phase

## adding to the computational graph

In [ ]:
# invoking tf.constant(), tf.matrix() etc. automatically adds to the graph. Assigning gives just a pointer
a = tf.constant(10)
assert a.graph is def_graph
b = tf.constant(10)
assert b.graph is def_graph

print(sess.run(a+b))
sum_ = a+b
print(sess.run(sum_))

ops = tf.get_default_graph().get_operations()

for i in ops:
    print(i is a)
    print(i.graph is def_graph)

### Let's try out making a second graph and adding stuff to it

In [ ]:
g = tf.Graph()
with g.as_default():
    c = tf.constant(10)
    assert c.graph is g
    
    

### Finalising graphs (making them read-only)

In [ ]:
g.finalize()

print('is g finalised?', g.finalized)
print('now try adding to g')
try:
    with g.as_default():
        d = tf.constant(10)
except: 
    print('didnt let us ...')

# Running phase

## simple example using variables

In [ ]:
# Create a Variable, that will be initialized to the scalar value 0.
state = tf.Variable(0, name="counter")

# Create an Op to add one to `state`.

one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

# Variables must be initialized by running an `init` Op after having
# launched the graph.  We first have to add the `init` Op to the graph.
init_op = tf.initialize_all_variables()

# Launch the graph and run the ops.
with tf.Session() as sess:
  # Run the 'init' op
  sess.run(init_op)
  # Print the initial value of 'state'
  print(sess.run(state))
  # Run the op that updates 'state' and print 'state'.
  for _ in range(3):
    sess.run(update)
    print(sess.run(state))


# An applied example ...

In [ ]:
%matplotlib notebook
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

def_graph = tf.get_default_graph() # doesn't do anything, but nice to know we can touch the graph
sess = tf.Session()

def matpow(M, n):
    if n < 1: #Abstract cases where n < 1
        return M
    else:
        return tf.matmul(M, matpow(M, n-1))

p, n = 20, 3
k, l = 3, 3

gamma = 1.

C_true = np.asarray(np.random.normal(size=(p,n)), dtype=np.float32)
B_true = np.asarray(np.random.normal(size=(n,n)), dtype=np.float32)
Pi_true = B_true.dot(B_true.T)

A_true = np.random.normal(size=(n,n))
A_true /= np.atleast_2d(np.sqrt(np.sum(A_true**2, axis=0)))
A_true = A_true.dot( np.diag( np.linspace(0.8, 1.05, n))).dot(np.linalg.inv(A_true))
A_true = np.asarray(A_true, dtype=np.float32)

A_0 = 0.9 * np.asarray(np.eye(n), dtype=np.float32)

print('sig(A_0)', np.linalg.svd(A_0)[1])
print('eig(A_0)', np.linalg.eigvals(A_0))

C  = tf.Variable(initial_value=tf.random_normal(shape=(p,n)), name='C')  # note how we may not use 
A  = tf.Variable(initial_value=A_0, name='A')                            # the tf.constants p_, n_.
Pi = tf.Variable(initial_value=tf.random_normal(shape=(n,n)), name='Pi') # tf wants fixed (numpy) numbers

loss = tf.zeros(1, name ='loss_m'+str(0))
Qm_models, Qm_emps = [], []

for m in range(1,k+l):
    Qm_models.append(tf.matmul(tf.matmul(C, 
                                         tf.matmul( matpow(A, m), 
                                                    Pi)), 
                               tf.transpose(C), 
                     name='Qm_model'+str(m)))
    Qm_emps.append(  tf.matmul(tf.matmul(C_true, 
                                         tf.matmul( matpow(A_true, m), 
                                                    Pi_true)), 
                               tf.transpose(C_true), 
                   name='Q_emp'+str(m)))
    loss = tf.add(loss, 
                  tf.reduce_mean(tf.squared_difference(Qm_models[-1], 
                                                       Qm_emps[-1], 
                                                       name='SE'+str(m)), 
                                 name='MSE'+str(m)), 
                  name ='loss_m'+str(m))
    
loss_barrier = tf.log( tf.matrix_determinant( tf.sub(np.asarray(np.eye(n),dtype=np.float32), 
                                                     tf.matmul(A, tf.transpose(A)) ) ),
                       name='loss_barrer')
loss_final = tf.sub(loss, tf.mul(tf.constant(gamma) , loss_barrier), name='loss_final')

loss2use = loss_final

optimizer = tf.train.GradientDescentOptimizer(0.00001)

global_step = tf.Variable(0, name='global_step', trainable=False)
train_op = optimizer.minimize(loss2use, global_step=global_step)

init = tf.initialize_all_variables()

#tf.scalar_summary(loss.op.name, loss)
#summary_op = tf.merge_all_summaries()
#summary_writer = tf.train.SummaryWriter(FLAGS.train_dir, sess.graph)

sess.run(init)

err = np.sum([np.mean( (sess.run(Qm_emps[m]) - sess.run(Qm_models[m]))**2 ) for m in range(k+l-1)]) 
bar = gamma * np.log( np.linalg.det( np.eye(n) - sess.run(A).dot(sess.run(A).T)))
print('initial error', err - bar)
print('(and what tf thinks it was):', sess.run(loss2use) )

In [ ]:


max_iter = 10000
loss_values = np.zeros(max_iter)

print('pre training')
plt.figure(figsize=(12,8))
for m in range(1,k+l):
    plt.subplot(2,k+l-1,m)
    plt.imshow(sess.run(Qm_emps[m-1]), interpolation='none')
    plt.subplot(2,k+l-1,k+l-1 + m)
    plt.imshow(sess.run(Qm_models[m-1]), interpolation='none')
plt.show()

for t in range(max_iter):
    _, loss_values[t] = sess.run([train_op, loss2use])

plt.figure(figsize=(12,5))
plt.plot(loss_values)
plt.show()

print('post training')
plt.figure(figsize=(12,8))
for m in range(1,k+l):
    plt.subplot(2,k+l-1,m)
    plt.imshow(sess.run(Qm_emps[m-1]), interpolation='none')
    plt.subplot(2,k+l-1,k+l-1 + m)
    plt.imshow(sess.run(Qm_models[m-1]), interpolation='none')
plt.show()

print('post-training error', 
      np.sum([np.mean( (sess.run(Qm_emps[m]) - sess.run(Qm_models[m]))**2 ) for m in range(k+l-1)]) )
print('(and what tf thinks it was):', sess.run(loss) )


print('current barrier value', sess.run(loss_barrier))
print('actual barrier value',  
      gamma * np.log( np.linalg.det( np.eye(n) - sess.run(A).dot(sess.run(A).T)) ))

print('true spectrum', np.linalg.svd(A_true)[1])
print('est. spectrum', np.linalg.svd(sess.run(A))[1])

